ALS + GB
Подобрать другие retrivials

In [1]:
%autosave 60
%reload_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "..")  # noqa: E702

Autosaving every 60 seconds


In [2]:
from catboost.utils import get_gpu_device_count

get_gpu_device_count()

1

In [3]:
import implicit.gpu
import polars as pl
import numpy as np
import implicit
from scipy.sparse import csr_matrix
from pathlib import Path
from catboost import CatBoostRanker, Pool
from datetime import timedelta


implicit.gpu.HAS_CUDA
# from tools.experiment_tracker import ExperimentTracker

/home/laptopml/avito_cup_ml_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
from pathlib import Path

DATA_DIR = Path.cwd().parent / "data" / "avito_ml_cup"
# Разделение на три части: тренировочную, валидационную и тестовую
VALID_DAYS_RETRIEVER = 7
VALID_DAYS_RERANKER = 0

In [5]:
df_test_users = pl.read_parquet(f"{DATA_DIR}/test_users.pq")
df_clickstream = pl.read_parquet(f"{DATA_DIR}/clickstream.pq")

df_cat_features = pl.read_parquet(f"{DATA_DIR}/cat_features.pq", n_rows=100_000)
df_text_features = pl.read_parquet(f"{DATA_DIR}/text_features.pq", n_rows=100_000)
df_event = pl.read_parquet(f"{DATA_DIR}/events.pq")

In [6]:
df_clickstream

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,4247649,17,2025-02-17 14:55:17,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458
…,…,…,…,…,…,…
149999,4999183,17,2025-01-20 12:23:47,2,2,71511
149999,25999164,17,2025-01-24 14:26:57,2,2,71514
149999,12138732,17,2025-02-12 13:11:42,2,2,51162


In [7]:
df_clickstream["event_date"].max() - df_clickstream["event_date"].min()

datetime.timedelta(days=44)

In [8]:
# Определяем границы для разделения данных
max_date = df_clickstream["event_date"].max()
retrivial_threshold = max_date - timedelta(days=VALID_DAYS_RETRIEVER + VALID_DAYS_RERANKER)
reranker_threshold = max_date - timedelta(days=VALID_DAYS_RERANKER)

# Разделяем данные
df_train_retrivial = df_clickstream.filter(pl.col("event_date") < retrivial_threshold)
df_train_reranker = df_clickstream.filter(
    pl.col("event_date").is_between(retrivial_threshold, reranker_threshold)
)
df_valid = df_clickstream.filter(pl.col("event_date") > reranker_threshold)


# Создаем оценочные наборы для валидации и тестирования
def prepare_eval_set(df_eval_raw, df_train_data):
    # Выбираем только нужные колонки
    df_eval = df_eval_raw[["cookie", "node", "event"]]

    # Исключаем пары cookie-node, которые уже встречались в тренировочных данных
    df_eval = df_eval.join(df_train_data, on=["cookie", "node"], how="anti")

    # Оставляем только контактные события
    df_eval = df_eval.filter(
        pl.col("event").is_in(
            df_event.filter(pl.col("is_contact") == 1)["event"].unique()
        )
    )

    # Оставляем только существующие в тренировочных данных cookies и nodes
    df_eval = df_eval.filter(
        pl.col("cookie").is_in(df_train_data["cookie"].unique())
    ).filter(pl.col("node").is_in(df_train_data["node"].unique()))

    # Оставляем только уникальные пары cookie-node
    df_eval = df_eval.unique(["cookie", "node"])

    return df_eval


# Подготавливаем валидационный и тестовый наборы
df_valid_eval_retrivial = prepare_eval_set(df_train_reranker, df_train_retrivial)
df_valid_eval_reranker = prepare_eval_set(
    df_valid, pl.concat([df_train_retrivial, df_train_reranker])
)

df_train_reranker = df_train_reranker.join(df_event, on="event")

df_valid = df_valid.join(df_event, on="event")

/tmp/ipykernel_63030/3723676151.py:23: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_63030/3723676151.py:30: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_63030/3723676151.py:32: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).filter(pl.col("node").is_in(df_train_data["node"].unique()))


In [9]:
df_valid_eval_reranker.join(df_event, on="event").sort("cookie")

cookie,node,event,is_contact
i64,u32,i64,i64


In [10]:
# Выводим информацию о размерах наборов данных
print(f"Размер тренировочного набора retrivial: {df_train_retrivial.shape}")
print(f"Размер валидационного набора reranker: {df_train_reranker.shape}")
print(f"Размер валидационного набора: {df_valid.shape}")
print(
    f"Размер валидационного набора для оценки retrivial: {df_valid_eval_retrivial.shape}"
)
print(
    f"Размер валидационного набора для оценки reranker: {df_valid_eval_reranker.shape}"
)

Размер тренировочного набора retrivial: (57032043, 7)
Размер валидационного набора reranker: (11774109, 8)
Размер валидационного набора: (0, 8)
Размер валидационного набора для оценки retrivial: (74771, 3)
Размер валидационного набора для оценки reranker: (0, 3)


# Train model

## Train ALS

In [11]:
class ALSRetriever:
    def __init__(self, **params_als):
        self.model = implicit.als.AlternatingLeastSquares(**params_als)

        self._user_id_to_index: dict[int, int] | None = None
        self._item_id_to_index: dict[int, int] | None = None
        self._index_to_item_id: dict[int, int] | None = None
        self._sparse_matrix = None

    def _create_mappings(self, user_ids, item_ids):
        """Создает маппинги между ID и индексами для пользователей и товаров"""
        self._user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
        self._item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
        self._index_to_item_id = {v: k for k, v in self._item_id_to_index.items()}
        self._index_to_user_id = {v: k for k, v in self._user_id_to_index.items()}

    def fit(
        self,
        users,
        nodes,
        events=None,
        event_weights=None,
        contact_event_boost=10.0,
        contact_events=None,
        is_contact_col=None,
    ):
        """
        Обучает модель ALS

        Args:
            users (pl.Series): ID пользователей
            nodes (pl.Series): ID элементов (товаров)
            events (pl.Series, optional): Типы событий
            event_weights (dict, optional): Словарь весов для разных типов событий
            contact_event_boost (float): Множитель для контактных событий
            contact_events (list, optional): Список ID контактных событий
            is_contact_col (pl.Series, optional): Колонка с флагом контакта

        Returns:
            self: Возвращает self для возможности цепочки вызовов
        """
        user_ids = users.unique().to_list()
        item_ids = nodes.unique().to_list()

        self._create_mappings(user_ids, item_ids)

        rows = users.replace_strict(self._user_id_to_index).to_list()
        cols = nodes.replace_strict(self._item_id_to_index).to_list()

        # Определение весов взаимодействий
        if events is not None and event_weights:
            values = [event_weights.get(event, 1.0) for event in events.to_list()]

            # Если указаны контактные события, увеличиваем их вес
            if contact_events:
                values = [
                    val * contact_event_boost if events[i] in contact_events else val
                    for i, val in enumerate(values)
                ]
        elif is_contact_col is not None:
            # Если есть прямой признак контакта, используем его для весов
            values = [
                contact_event_boost if is_contact else 1.0
                for is_contact in is_contact_col.to_list()
            ]
        else:
            # По умолчанию все взаимодействия имеют вес 1
            values = [1] * len(users)

        # Создаем разреженную матрицу взаимодействий
        self._sparse_matrix = csr_matrix(
            (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
        )

        # Обучаем модель
        self.model.fit(self._sparse_matrix)

    def recommend(self, user_ids, n=40, filter_already_liked_items=True):
        """
        Генерирует рекомендации для указанных пользователей

        Args:
            user_ids (list): Список ID пользователей для рекомендаций
            n (int): Количество рекомендаций для каждого пользователя
            filter_already_liked_items (bool): Исключать ли уже взаимодействовавшие товары
            recalculate_user (bool): Пересчитывать ли модель для пользователя

        Returns:
            pl.DataFrame: DataFrame с рекомендациями
        """
        if not self._user_id_to_index:
            raise ValueError("Сначала надо fit.")
        # Преобразуем ID пользователей в индексы
        user_indices = []
        valid_users = []

        for user_id in user_ids:
            if user_id in self._user_id_to_index:
                user_indices.append(self._user_id_to_index[user_id])  # type: ignore
                valid_users.append(user_id)

        if not user_indices:
            return pl.DataFrame({"cookie": [], "node": [], "scores": []})

        # Получаем рекомендации
        user_index_array = np.array(user_indices)
        recommendations, scores = self.model.recommend(
            user_index_array,
            self._sparse_matrix[user_index_array],  # type: ignore
            N=n,
            filter_already_liked_items=filter_already_liked_items,
        )

        # Преобразуем индексы товаров обратно в ID
        node_lists = [
            [self._index_to_item_id[idx] for idx in rec] for rec in recommendations
        ]  # type: ignore

        # Создаем DataFrame с результатами
        df_pred = pl.DataFrame(
            {"node": node_lists, "cookie": valid_users, "scores": scores.tolist()}
        )

        # Разворачиваем списки
        df_pred = df_pred.explode(["node", "scores"])

        return df_pred

    def save_model(self, path: str):
        import pickle

        model_data = {
            "model": self.model,
            "user_id_to_index": self._user_id_to_index,
            "item_id_to_index": self._item_id_to_index,
            "index_to_item_id": self._index_to_item_id,
            "index_to_user_id": self._index_to_user_id,
            "sparse_matrix": self._sparse_matrix,
        }

        with open(path, "wb") as f:
            pickle.dump(model_data, f)

    @classmethod
    def load(cls, path):
        """
        Загружает модель из файла

        Args:
            path (str): Путь к файлу модели

        Returns:
            ALSRetrivier: Загруженная модель
        """
        import pickle

        with open(path, "rb") as f:
            model_data = pickle.load(f)

        retriever = cls()
        retriever.model = model_data["model"]
        retriever._user_id_to_index = model_data["user_id_to_index"]
        retriever._item_id_to_index = model_data["item_id_to_index"]
        retriever._index_to_item_id = model_data["index_to_item_id"]
        retriever._index_to_user_id = model_data["index_to_user_id"]
        retriever._sparse_matrix = model_data["sparse_matrix"]

        return retriever


def get_als_pred(
    users,
    nodes,
    events,
    user_to_pred,
    event_weights=None,
    generate_candidates: int = 40,
):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()

    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}

    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()

    # Используем веса событий или 1 по умолчанию
    if event_weights is not None and events is not None:
        values = [event_weights.get(event, 1.0) for event in events.to_list()]
    else:
        values = [1] * len(users)

    sparse_matrix = csr_matrix(
        (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
    )

    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=100)
    model.fit(
        sparse_matrix,
    )

    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])

    recommendations, scores = model.recommend(
        user4pred,
        sparse_matrix[user4pred],
        N=generate_candidates,
        filter_already_liked_items=True,
    )

    df_pred = pl.DataFrame(
        {
            "node": [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ],
            "cookie": list(user_to_pred),
            "scores": scores.tolist(),
        }
    )
    df_pred = df_pred.explode(["node", "scores"])
    return df_pred

In [12]:
als_retriever = ALSRetriever(iterations=50, factors=250, regularization=0.3, use_gpu=True)

In [13]:
users = df_train_retrivial["cookie"]
nodes = df_train_retrivial["node"]
events = df_train_retrivial["event"]
eval_users = df_valid_eval_retrivial["cookie"].unique().to_list()

In [14]:
def create_event_weights(clickstream_df: pl.DataFrame):
    event_count = {
        row[0]: row[1] for row in clickstream_df.group_by("event").len().rows()
    }

    return {event_id: 1.0 / np.log1p(count) for event_id, count in event_count.items()}


inverse_freq_weights = create_event_weights(df_train_retrivial.select("event"))

# Можно также попробовать эту альтернативную стратегию взвешивания
# df_pred_alt = get_als_pred(users, nodes, events, eval_users, inverse_freq_weights, 100)

In [15]:
als_retriever.fit(
    users, nodes, events, inverse_freq_weights, 100, list(inverse_freq_weights.keys()))

100%|██████████| 50/50 [02:38<00:00,  3.17s/it]


In [16]:
df_pred_alt = als_retriever.recommend(eval_users, 200)

In [17]:
del als_retriever

In [18]:
class PopularItemsRetriever:
    """
    Класс для получения популярных товаров по различным критериям
    """

    def __init__(self, clickstream_df=None, categories_df=None):
        """
        Инициализирует ретривер популярных товаров

        Args:
            clickstream_df (pl.DataFrame, optional): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров
        """
        self.clickstream_df = clickstream_df
        self.categories_df = categories_df

        # Кэш для хранения результатов
        self._cache = {}

    def set_data(self, clickstream_df, categories_df=None):
        """
        Устанавливает данные для работы

        Args:
            clickstream_df (pl.DataFrame): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров

        Returns:
            self: Возвращает self для цепочки вызовов
        """
        self.clickstream_df = clickstream_df
        if categories_df is not None:
            self.categories_df = categories_df

        # Сбрасываем кэш при обновлении данных
        self._cache = {}

        return self

    def get_popular_items(self, days=7, top_k=100, use_cache=True):
        """
        Получает популярные товары за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами и количеством взаимодействий
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        cache_key = f"popular_{days}_{top_k}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Подсчитываем популярность каждого node
        popular_nodes = (
            recent_data.group_by("node")
            .agg(pl.count().alias("interaction_count"))
            .sort("interaction_count", descending=True)
            .head(top_k)
        )

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = popular_nodes

        return popular_nodes

    def get_popular_items_by_category(self, days=7, top_k=20, use_cache=True):
        """
        Получает популярные товары по категориям за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров на категорию
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами по категориям
        """
        if self.clickstream_df is None or self.categories_df is None:
            raise ValueError("Не установлены данные кликстрима или категорий товаров")

        cache_key = f"popular_by_category_{days}_{top_k}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Соединяем с категориями
        recent_with_cat = recent_data.join(
            self.categories_df[["node", "category"]].unique(), on="node", how="inner"
        )

        # Подсчитываем популярность по категориям
        popular_by_cat = (
            recent_with_cat.group_by(["category", "node"])
            .agg(pl.count().alias("interaction_count"))
            .sort(["category", "interaction_count"], descending=[False, True])
        )

        # Берем top_k для каждой категории
        result = popular_by_cat.group_by("category").head(top_k)

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = result

        return result

    def get_trending_items(self, days_window=3, previous_days=7, top_k=100):
        """
        Получает тренды - товары, растущие по популярности

        Args:
            days_window (int): Размер окна для текущей популярности (дни)
            previous_days (int): Количество дней для сравнения с предыдущим периодом
            top_k (int): Количество растущих товаров для возврата

        Returns:
            pl.DataFrame: Таблица с растущими товарами
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        max_date = self.clickstream_df["event_date"].max()

        # Данные для текущего окна
        current_cutoff = max_date - timedelta(days=days_window)
        current_data = self.clickstream_df.filter(pl.col("event_date") > current_cutoff)

        # Данные для предыдущего окна
        prev_start = current_cutoff - timedelta(days=previous_days)
        prev_data = self.clickstream_df.filter(
            (pl.col("event_date") <= current_cutoff)
            & (pl.col("event_date") > prev_start)
        )

        # Вычисляем популярность для текущего и предыдущего окна
        current_pop = current_data.group_by("node").agg(
            pl.count().alias("current_count")
        )

        prev_pop = prev_data.group_by("node").agg(pl.count().alias("previous_count"))

        # Объединяем и вычисляем рост
        trending = (
            current_pop.join(prev_pop, on="node", how="left")
            .with_columns(
                [
                    pl.col("previous_count").fill_null(0),
                    (pl.col("current_count") - pl.col("previous_count")).alias(
                        "growth"
                    ),
                    (
                        (pl.col("current_count") - pl.col("previous_count"))
                        / (pl.col("previous_count") + 1)
                    ).alias("growth_rate"),
                ]
            )
            .sort("growth_rate", descending=True)
            .head(top_k)
        )

        return trending

    def get_popular_for_user(self, user_id, days=30, top_k=50):
        """
        Получает популярные товары для конкретного пользователя

        Args:
            user_id: ID пользователя (cookie)
            days (int): Количество дней для анализа
            top_k (int): Количество товаров для возврата

        Returns:
            pl.DataFrame: Таблица с товарами, популярными среди похожих пользователей
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        # Здесь может быть реализована логика определения похожих пользователей
        # и получения популярных товаров среди них

        # Простая реализация: товары из категорий, с которыми взаимодействовал пользователь
        if self.categories_df is None:
            return self.get_popular_items(days, top_k)

        # Получаем категории товаров, с которыми взаимодействовал пользователь
        user_categories = (
            self.clickstream_df.filter(pl.col("cookie") == user_id)
            .join(
                self.categories_df[["node", "category"]].unique(),
                on="node",
                how="inner",
            )["category"]
            .unique()
        )

        if len(user_categories) == 0:
            return self.get_popular_items(days, top_k)

        # Получаем популярные товары из категорий пользователя
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Фильтруем по категориям пользователя
        user_cat_items = (
            recent_data.join(
                self.categories_df.filter(pl.col("category").is_in(user_categories))[
                    ["node", "category"]
                ].unique(),
                on="node",
                how="inner",
            )
            .group_by("node")
            .agg(pl.count().alias("interaction_count"))
            .sort("interaction_count", descending=True)
            .head(top_k)
        )

        return user_cat_items

In [19]:
popular_retriever = PopularItemsRetriever(df_train_retrivial, df_cat_features)

# Получаем популярные товары за последнюю неделю и месяц
popular_week = popular_retriever.get_popular_items(days=7, top_k=200)
popular_month = popular_retriever.get_popular_items(days=30, top_k=300)
popular_by_category = popular_retriever.get_popular_items_by_category(days=7, top_k=30)

/tmp/ipykernel_63030/1626469783.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))
/tmp/ipykernel_63030/1626469783.py:108: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))


# Двустадийная система рекомендаций
## Этап 1: ALS для первичного отбора релевантных товаров
## Этап 2: CatBoost для ранжирования и уточнения результатов

In [20]:
df_train_reranker

cookie,item,event,event_date,platform,surface,node,is_contact
i64,i64,i64,datetime[ns],i64,i64,u32,i64
1,4247649,17,2025-02-17 14:55:17,2,2,7,0
1,1874892,17,2025-02-22 07:30:38,2,2,243208,0
1,13385429,17,2025-02-16 14:33:07,2,2,151614,0
1,748003,17,2025-02-19 09:20:17,2,2,214364,0
1,6399591,17,2025-02-16 12:20:05,2,2,267694,0
…,…,…,…,…,…,…,…
149994,17225423,17,2025-02-18 17:11:24,2,2,199383,0
149994,17225423,17,2025-02-18 17:12:22,2,2,199383,0
149999,10685095,17,2025-02-19 13:06:25,2,2,136796,0


In [21]:
df_candidates_train = df_pred_alt.join(
    df_train_reranker["node", "cookie", "is_contact"]
    .sort("is_contact", descending=True)
    .unique(["cookie", "node"]),
    on=["cookie", "node"],
    how="left",
).fill_null(0)

In [22]:
# Подготовка данных для CatBoost
def prepare_features_for_boosting(
    df_retrieval,
    df_clickstream,
    df_cat_features,
    df_popular_week,
    df_popular_month,
    df_popular_by_cat,
):
    """Подготавливает признаки для обучения бустинга на основе результатов ALS"""
    # Добавляем информацию о рейтинге ALS
    features = df_retrieval.rename({"scores": "als_score"})

    # Добавляем признак о популярности группы товаров
    features = features.join(
        df_popular_week.select(["node", "interaction_count"]).rename(
            {"interaction_count": "popular_week_count"}
        ),
        on="node",
        how="left",
    ).with_columns(pl.col("popular_week_count").fill_null(-1))

    # # Добавляем признак о популярности группы товаров
    features = features.join(
        df_popular_month.select(["node", "interaction_count"]).rename(
            {"interaction_count": "popular_month_count"}
        ),
        on="node",
        how="left",
    ).with_columns(pl.col("popular_month_count").fill_null(-1))

    # Добавляем количество взаимодействий пользователя
    user_activity = df_clickstream.group_by("cookie").agg(
        pl.len().alias("user_activity_count")
    )
    features = features.join(user_activity, on="cookie", how="left")

    # Добавляем информацию о категории товара
    features = features.join(
        df_cat_features.select(["node", "category"]).unique(),
        on="node",
        how="left",
    ).fill_null(-1)

    features = features.join(
        df_popular_by_cat, on=["node", "category"], how="left"
    ).fill_null(-1)

    return features


# Подготовка данных для обучения CatBoost
def prepare_catboost_data(features_df, df_eval, max_items_per_user=100):
    """Подготавливает данные для обучения CatBoost с позитивными и негативными примерами"""
    # Фильтруем топ-N предсказаний ALS для каждого пользователя
    top_predictions = features_df.group_by("cookie")

    # Помечаем позитивные примеры (те, которые есть в df_eval)
    labeled_data = top_predictions.join(
        df_eval[["cookie", "node"]], on=["cookie", "node"], how="left"
    )

    return labeled_data


# Готовим данные для CatBoost
train_data_for_boosting = prepare_features_for_boosting(
    df_candidates_train,
    df_train_retrivial,
    df_cat_features,
    popular_week,
    popular_month,
    popular_by_category,
)

In [23]:
train_data_for_boosting

node,cookie,als_score,is_contact,popular_week_count,popular_month_count,user_activity_count,category,interaction_count
i64,i64,f64,i64,i64,i64,i64,i64,i64
115834,0,0.955596,0,8004,37031,258,19,8004
115669,0,0.921513,0,-1,-1,258,19,-1
1907,0,0.920921,0,18366,55094,258,25,18366
115810,0,0.9163,0,-1,-1,258,19,4729
1923,0,0.890081,0,23326,89071,258,61,23326
…,…,…,…,…,…,…,…,…
214290,149997,0.389333,0,12345,52178,284,37,12345
132695,149997,0.388594,0,-1,-1,284,24,-1
134516,149997,0.388434,0,-1,-1,284,24,-1


In [24]:
feature_cols = [
    "node",
    "als_score",
    "popular_week_count",
    "popular_month_count",
    "user_activity_count",
    "category",
    "interaction_count"
]

target_col = "is_contact"

In [25]:
model = CatBoostRanker(
    boosting_type="Plain",
    iterations=500,
    learning_rate=0.05,
    depth=8,
    objective="PairLogitPairwise:max_pairs=100000",
    random_seed=42,
    verbose=50,
    nan_mode="Min",
    task_type="GPU"
)

In [26]:
model.fit(
    train_data_for_boosting.select(feature_cols).to_pandas(),
    train_data_for_boosting.select(target_col).to_pandas(),
    group_id=train_data_for_boosting["cookie"].to_list(),
)

0:	learn: 0.6831864	total: 450ms	remaining: 3m 44s
50:	learn: 0.5598720	total: 15.1s	remaining: 2m 13s
100:	learn: 0.5476751	total: 29.3s	remaining: 1m 55s
150:	learn: 0.5410434	total: 42.8s	remaining: 1m 38s
200:	learn: 0.5361645	total: 56.3s	remaining: 1m 23s
250:	learn: 0.5316450	total: 1m 9s	remaining: 1m 9s
300:	learn: 0.5275545	total: 1m 23s	remaining: 55.3s
350:	learn: 0.5240518	total: 1m 38s	remaining: 41.7s
400:	learn: 0.5205834	total: 1m 51s	remaining: 27.6s
450:	learn: 0.5173379	total: 2m 5s	remaining: 13.6s
499:	learn: 0.5144240	total: 2m 19s	remaining: 0us


# Оценка моделей
## Метрика Recall@K

In [27]:
def recall_at(df_true, df_pred, k=40):
    """Вычисляет метрику Recall@K для рекомендаций"""
    return (
        df_true[["node", "cookie"]]
        .join(
            df_pred.group_by("cookie")
            .head(k)
            .with_columns(value=1)[["node", "cookie", "value"]],
            how="left",
            on=["cookie", "node"],
        )
        .select([pl.col("value").fill_null(0), "cookie"])
        .group_by("cookie")
        .agg([pl.col("value").sum() / pl.col("value").count()])["value"]
        .mean()
    )


def evaluate_models(
    df_true, df_als_pred, df_catboost_pred=None, k_values=[10, 20, 30, 40]
):
    """Сравнивает модели по метрике Recall@K для разных значений K"""
    results = {}

    for k in k_values:
        results[f"ALS_recall@{k}"] = recall_at(df_true, df_als_pred, k=k)
        if df_catboost_pred is not None:
            results[f"CatBoost_recall@{k}"] = recall_at(df_true, df_catboost_pred, k=k)

    return results


# Оцениваем модель ALS на валидационном наборе
als_results = evaluate_models(
    df_valid_eval_retrivial, df_pred_alt, k_values=[10, 20, 30, 40]
)
print("\nРезультаты ALS модели на валидационном наборе:")
for metric, value in als_results.items():
    print(f"{metric}: {value:.4f}")



Результаты ALS модели на валидационном наборе:
ALS_recall@10: 0.0651
ALS_recall@20: 0.1027
ALS_recall@30: 0.1302
ALS_recall@40: 0.1522


In [28]:
# recall_at(df_eval, df_pred_alt, k=40)

# Можно сохранить обученные модели и результаты для дальнейшего использования

# Пример организации модулей для ClearML:
"""
from tools.experiment_tracker import ExperimentTracker

# Инициализация трекера экспериментов
tracker = ExperimentTracker(project_name="avito_cup_ml_2025", task_name="als_catboost_pipeline")

# Логирование параметров
tracker.log_params({
    'als_factors': 60,
    'als_iterations': 10,
    'catboost_iterations': 500,
    'catboost_learning_rate': 0.05,
    'catboost_depth': 6
})

# Логирование метрик
tracker.log_metrics(als_results)

# Сохранение моделей
tracker.save_model(model, 'catboost_ranker')

# Завершение эксперимента
tracker.close()
"""


df_train_reranker = df_train_reranker.drop("is_contact")
df_train_retriever_and_reranker = pl.concat([df_train_retrivial, df_train_reranker])

# Применение двустадийной модели на тестовом наборе
# Сначала получаем рекомендации от ALS

# test_users = df_valid_eval_reranker["cookie"].unique().to_list()
test_users = df_test_users["cookie"].unique().to_list()


# Объединяем тренировочные и валидационные данные для финального обучения
full_train_users = df_train_retriever_and_reranker["cookie"]
full_train_nodes = df_train_retriever_and_reranker["node"]
full_train_events = df_train_retriever_and_reranker["event"]

# Создаем веса на основе обратной частоты и контактности
inverse_freq_weights = create_event_weights(df_train_retriever_and_reranker.select("event"))

In [29]:
als_retriever = ALSRetriever(iterations=50, factors=250, regularization=0.3)

als_retriever.fit(full_train_users, full_train_nodes, full_train_events, inverse_freq_weights, 100, list(inverse_freq_weights.keys()))

# Получаем предсказания ALS для тестового набора
test_als_pred = als_retriever.recommend(test_users, 200)

100%|██████████| 50/50 [02:56<00:00,  3.52s/it]


In [30]:
popular_retriever = PopularItemsRetriever(df_train_retriever_and_reranker, df_cat_features)

# Получаем популярные товары за последнюю неделю и месяц
popular_week = popular_retriever.get_popular_items(days=7, top_k=200)
popular_month = popular_retriever.get_popular_items(days=30, top_k=300)
popular_by_category = popular_retriever.get_popular_items_by_category(days=7, top_k=30)

/tmp/ipykernel_63030/1626469783.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))
/tmp/ipykernel_63030/1626469783.py:108: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))


In [31]:
test_data_for_boosting = prepare_features_for_boosting(
    test_als_pred,
    df_train_retriever_and_reranker,
    df_cat_features,
    popular_week,
    popular_month,
    popular_by_category,
)

In [32]:
test_data_for_boosting

node,cookie,als_score,popular_week_count,popular_month_count,user_activity_count,category,interaction_count
i64,i64,f64,i64,i64,i64,i64,i64
1928,1,1.42856,-1,23214,4590,13,5696
5744,1,1.344092,18225,74008,4590,28,18225
214272,1,1.309013,-1,-1,4590,37,-1
153004,1,1.302189,-1,25366,4590,40,-1
214232,1,1.228835,-1,-1,4590,37,-1
…,…,…,…,…,…,…,…
151580,149999,0.428171,-1,-1,785,40,-1
51208,149999,0.427191,-1,-1,785,49,2918
115834,149999,0.426486,9491,38030,785,19,9491


In [33]:
df_pred_catboost = (
    pl.DataFrame(
        {
            "node": test_data_for_boosting["node"],
            "cookie": test_data_for_boosting["cookie"],
            "catboost_score": model.predict(test_data_for_boosting.to_pandas()),
        }
    )
    .sort(by=["cookie", "catboost_score"], descending=True)
    .group_by("cookie")
    .head(40)
)

In [34]:
df_pred_catboost[["cookie", "node"]].write_csv("../data/predict/baseline_als_catboost.csv")

In [40]:
sorted(df_pred_catboost["cookie"].unique().to_list()) == sorted(test_users)

True

In [39]:
df_valid_eval_reranker

cookie,node,event
i64,u32,i64
116029,188671,10
54368,194766,4
103610,152470,10
94011,214295,10
31901,130860,10
…,…,…
58959,122420,10
117281,14048,15
103542,201575,4


In [40]:
test_als_results = evaluate_models(
    df_valid_eval_reranker, test_als_pred, df_pred_catboost, k_values=[10, 20, 30, 40]
)
print("\nРезультаты ALS модели на тестовом наборе:")
for metric, value in test_als_results.items():
    print(f"{metric}: {value:.4f}")


Результаты ALS модели на тестовом наборе:
ALS_recall@10: 0.0661
CatBoost_recall@10: 0.0738
ALS_recall@20: 0.1026
CatBoost_recall@20: 0.1148
ALS_recall@30: 0.1303
CatBoost_recall@30: 0.1455
ALS_recall@40: 0.1533
CatBoost_recall@40: 0.1694
